# Cancer Detection using Resnet 50 
----

In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
import torchvision.models as models
import torchvision.transforms as transforms

## Load in the datasets
----
Info on how to obtain the datasets is contained in the README.md. Data augmentation techniques are applied in the data loader here to be applied to the training data. 

In [2]:
### Write data loaders for training, validation, and test sets
## Specify appropriate transforms, and batch_sizes
train_transfoms = transforms.Compose([transforms.Resize((224 , 224))
                              ,transforms.RandomRotation(degrees = 10)
                              ,transforms.RandomHorizontalFlip(p=0.2)
                              ,transforms.RandomGrayscale(p=0.2)
                              ,transforms.RandomVerticalFlip(p=0.2)
                              ,transforms.ToTensor()
                              ,transforms.Normalize(mean=[0.485, 0.456, 0.406]
                                                   , std=[0.229, 0.224, 0.225])
                              ])

test_val_transfomrs = transforms.Compose([transforms.Resize((224 , 224))
                              ,transforms.ToTensor()
                              ,transforms.Normalize(mean=[0.485, 0.456, 0.406]
                                                   , std = [0.229, 0.224, 0.225])
                              ])

batch_size = 4 #About all my little GPU can handle 
num_workers = 0

train_data = datasets.ImageFolder(root=r'C:\Users\diarm\Downloads\cancer_detection\data\train'
                                  , transform=train_transfoms)
test_data = datasets.ImageFolder(root=r'C:\Users\diarm\Downloads\cancer_detection\data\test'
                                 , transform=test_val_transfomrs)
valid_data = datasets.ImageFolder(root=r'C:\Users\diarm\Downloads\cancer_detection\data\valid'
                                  , transform=test_val_transfomrs)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)

loaders = dict(train=train_loader, test=test_loader, valid=valid_loader)

#### There are three class of skin lesion to look at in the problem.
    - melanoma
    - nevus
    - seborrheic_keratosis

In [3]:
!ls C:\\Users\\diarm\\Downloads\\cancer_detection\\data\\train

melanoma
nevus
seborrheic_keratosis


In [4]:
resnet50 = models.resnet50(pretrained=True)

# check if CUDA is available
use_cuda = torch.cuda.is_available()

# move model to GPU if CUDA is available
if use_cuda:
    resnet50 = resnet50.cuda()

#### The final classifier can be viewed here. 
For transfer learning we will change this final layer to suit our case and retrain it with our data. 

# Transfer learning
----
A new fully connected layer is appended to the ResNet50 structure. I'm going to use the underlying architecture of the network to piggyback on the extraction of features from an image, and train the parameters to suit our images. 

The ResNet-50 network was retrieved from pytorch.models - [you can find it here](https://pytorch.org/docs/stable/torchvision/models.html#torchvision.models.resnet18)


In [5]:
### Write data loaders for training, validation, and test sets
## Specify appropriate transforms, and batch_sizes
train_transfoms = transforms.Compose([transforms.Resize((224 , 224))
                              ,transforms.RandomRotation(degrees = 10)
                              ,transforms.RandomHorizontalFlip(p=0.2)
                              ,transforms.RandomGrayscale(p=0.2)
                              ,transforms.RandomVerticalFlip(p=0.2)
                              ,transforms.ToTensor()
                              ,transforms.Normalize(mean=[0.485, 0.456, 0.406]
                                                   , std=[0.229, 0.224, 0.225])
                              ])

test_val_transfomrs = transforms.Compose([transforms.Resize((224 , 224))
                              ,transforms.ToTensor()
                              ,transforms.Normalize(mean=[0.485, 0.456, 0.406]
                                                   , std = [0.229, 0.224, 0.225])
                              ])

batch_size = 4 #About all my little GPU can handle 
num_workers = 0

train_data = datasets.ImageFolder(root=r'C:\Users\diarm\Downloads\cancer_detection\data\train'
                                  , transform=train_transfoms)
test_data = datasets.ImageFolder(root=r'C:\Users\diarm\Downloads\cancer_detection\data\test'
                                 , transform=test_val_transfomrs)
valid_data = datasets.ImageFolder(root=r'C:\Users\diarm\Downloads\cancer_detection\data\valid'
                                  , transform=test_val_transfomrs)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)

loaders = dict(train=train_loader, test=test_loader, valid=valid_loader)

#### There are three class of skin lesion to look at in the problem.
    - melanoma
    - nevus
    - seborrheic_keratosis

In [6]:
!ls C:\\Users\\diarm\\Downloads\\cancer_detection\\data\\train

melanoma
nevus
seborrheic_keratosis


In [7]:
resnet50 = models.resnet50(pretrained=True)

# check if CUDA is available
use_cuda = torch.cuda.is_available()

# move model to GPU if CUDA is available
if use_cuda:
    resnet50 = resnet50.cuda()

In [8]:
# Two final fully connected layer for output
classifier = nn.Sequential(nn.Linear(2048, 512)
                           ,nn.ReLU()
                           ,nn.Dropout(0.15)
                           ,nn.Linear(512, 64)
                           ,nn.ReLU()
                           ,nn.Dropout(0.15)
                           ,nn.Linear(64, 3)
                          )
resnet50.fc = classifier
# Transfer to GPU if available
if use_cuda:
    resnet50 = resnet50.to('cuda')

In [9]:
resnet50.fc

Sequential(
  (0): Linear(in_features=2048, out_features=512, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.15, inplace=False)
  (3): Linear(in_features=512, out_features=64, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.15, inplace=False)
  (6): Linear(in_features=64, out_features=3, bias=True)
)

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet50.parameters(), lr=0.001)

In [11]:
# A helper function to show classification accuracy
def label_accuracy(model, validation_loader, epoch, use_cuda):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, targets in validation_loader:
            if use_cuda:
                images, targets = images.cuda(), targets.cuda()
            outputs = model(images)
            _, preds = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (preds == targets).sum().item()
    print('Accuray test at epoch {}\tLabel Accuracy : - {} %'.format(epoch, round(correct / total, 2)))

# Training 
----

In [12]:
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            # clear the accumulated gradients 
            optimizer.zero_grad()
            
            # inception net produces two outputs, aux output handles vanishing gradient
            outputs = model(data)
            #loss calculation https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
            loss = criterion(outputs, target)
            
            # gradient of the loss with respect to the parameters
            loss.backward()
            # perform the parameter update (update the weights)
            optimizer.step()
            
            ## record the average training loss
            # train loss - adds current loss to accumulated loss (averaged over batch size)
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
        
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            # get prediction
            output = model(data)
            # calculate the loss
            loss = criterion(output, target)
            ## update the average validation loss
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
            
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        ## save the model if validation loss has decreased
        if valid_loss < valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
            torch.save(model.state_dict(), save_path) #add save path here
            valid_loss_min = valid_loss
            # get epoch accuracy 
            #label_accuracy(model, loaders['valid'], epoch, use_cuda)
        
        # learning rate decay scheduled here if necessary
        #scheduler.step()
        
    return model

In [1]:
# let 'er riiiip
trained_model = train(50, loaders, resnet50, optimizer, criterion, use_cuda, save_path='./resnet_temp_models/model_cancer_detection.pt')

NameError: name 'train' is not defined